<section style="background-color: #21222c; padding: 1em">
    <h1><img alt="QuestDB Logo" src="https://play.questdb.io/questdb-logo.svg" width="305px"/></h1>
    <h2 style="color: white;">Interactive Notebook Session</h2>
</section>

In [1]:
import questdb.ingress as qi
import pandas as pd
import psycopg

In [2]:
# This demo relies on dynamic network ports for the core endpoints.
http_port = 9000  # Web Console and REST API
ilp_port = 9009  # Fast data ingestion port
pg_port = 8812  # PostgreSQL-compatible endpoint

In [3]:
df = pd.DataFrame({'a': [1, 2, 3, 4, 5]})

In [5]:
with qi.Sender('127.0.0.1', ilp_port) as sender:
    sender.dataframe(df, table_name='foo')

In [7]:
params = dict(user='admin', password='quest', host='127.0.0.1', port=pg_port, dbname='qdb')
records = None
with psycopg.connect(**params) as conn:
    with conn.cursor() as cur:
        cur.execute('SELECT * FROM foo')
        records = cur.fetchall()
records

[(1, datetime.datetime(2023, 1, 26, 13, 46, 7, 442909)),
 (2, datetime.datetime(2023, 1, 26, 13, 46, 7, 444148)),
 (3, datetime.datetime(2023, 1, 26, 13, 46, 7, 444153)),
 (4, datetime.datetime(2023, 1, 26, 13, 46, 7, 444156)),
 (5, datetime.datetime(2023, 1, 26, 13, 46, 7, 444159))]